In [1]:
import pandas as pd

In [2]:
from fast_pandas import FastPandas

In [3]:
df = pd.DataFrame({"a": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

In [4]:
a = FastPandas(df)["a"]
print(a)

"a"


In [5]:
a = FastPandas(df)["a"]

In [8]:
a.mul(3).sum().item()

165.0

# Use hyperloglog to count unique items

In [46]:
import random
df = pd.DataFrame({"items": [random.randint(0, 10_000_000) for _ in range(100_000_000)]})

In [47]:
len(df["items"].unique())

9999538

In [48]:
FastPandas(df)["items"].approx_count_distinct().item()

10037331

## Entropy

In [49]:
FastPandas(df)["items"].entropy().item()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

23.179988867936974